In [ ]:
from a1chemy.data_source import SinaFinanceClient
from a1chemy.common import OptionConfig
from a1chemy.option.pricing import black_scholes, black_76, gbs

config = OptionConfig(interest_rate=0.029120, pricing_model=gbs)
s = SinaFinanceClient()
m = s.get_option_map(symbol='SH510300', year=21, month=9, config=config)
for expire_date, chain in m.index.items():
    print(expire_date)
    for strike, straddle in chain.index.items():
        print('{} {} {} {}'.format(expire_date, straddle.strike, straddle.call.price, straddle.put.price))

In [ ]:
import datetime

expire_date = datetime.date.fromisoformat('2021-09-22')
o = m.get_option(expire_date, strike=4.5, option_type='c')
print(o.config.pricing_model.__name__)
print(o.config.interest_rate)
o.toString()